In [3]:
from sklearn.datasets import load_wine
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)
from matplotlib import pyplot as plt
import seaborn as sns
sns.set()
from sklearn.preprocessing import LabelEncoder
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Create Linear Discriminant Analysis From Scratch

Use sklearn Wine dateset to run an LDA algo against

In [4]:
wine = load_wine()
X = pd.DataFrame(wine.data, columns=wine.feature_names)
y = pd.Categorical.from_codes(wine.target, wine.target_names)

In [5]:
X.shape

(178, 13)

In [8]:
# create a dataframe with both features and classes

df = X.join(pd.Series(y, name='wine'))
df.head()

,alcohol,malic_acid,ash,alcalinity_of_ash,magnesium,total_phenols,flavanoids,nonflavanoid_phenols,proanthocyanins,color_intensity,hue,od280/od315_of_diluted_wines,proline,wine
0,14.23,1.71,2.43,15.6,127.0,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065.0,class_0
1,13.20,1.78,2.14,11.2,100.0,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050.0,class_0
2,13.16,2.36,2.67,18.6,101.0,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185.0,class_0
3,14.37,1.95,2.50,16.8,113.0,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480.0,class_0
4,13.24,2.59,2.87,21.0,118.0,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735.0,class_0


# Linear Discriminant Analysis Steps

https://towardsdatascience.com/linear-discriminant-analysis-in-python-76b8b17817c2

Linear Discriminant Analysis can be broken up into the following steps:
- Compute the within class and between class scatter matrices
- Compute the eigenvectors and corresponding eigenvalues for the scatter matrices
- Sort the eigenvalues and select the top k
- Create a new matrix containing eigenvectors that map to the k eigenvalues
- Obtain the new features (i.e. LDA components) by taking the dot product of the data and the matrix from step 4

Linear discriminant analysis of the form discussed above has its roots in an approach developed by the famous statistician R.A. Fisher, who arrived at linear discriminants from a different perspective. He was interested in finding a linear projection for data that maximizes the variance between classes relative to the variance for data from the same class. This approach is known as Fisher’s linear discriminant analysis, and can be formulate for two classes or multiple classes.

# Within Class Scatter Matrix
We calculate the within class scatter matrix using the following formula.

A scatter matrix is a statistic that is used to make estimates of the covariance matrix, for instance of the multivariate normal distribution.

Lets define the scatter matrix (within) $S_W$ as

$$S_W = \sum_{i=1}^{c}{S_i}$$

where c is the total number of distinct classes and

$$S_i = \sum_{x\in D_i}^{c}{(x-m_i)(x-m_i)^T}$$

$$m_i = \frac{1}{n_i} \sum_{x\in D_i}^{n}{x_k}$$

where x is a sample (i.e. row) and n is the total number of samples with a given class.

For every class, we create a vector with the means of each feature.

The full equation can be written as

$$S_W = \sum_{i=1}^{c}{\sum_{x\in D_i}^{c}{(x-m_i)(x-m_i)^T}}$$

$$S_W = \sum_{i=1}^{c}{\sum_{x\in D_i}^{c}{(x-\frac{1}{n_i} \sum_{x\in D_i}^{n}{x_k})(x-\frac{1}{n_i} \sum_{x\in D_i}^{n}{x_k})^T}}$$


In [6]:
class_feature_means = pd.DataFrame(columns=wine.target_names)
class_feature_means

,class_0,class_1,class_2


# STEP 1 - Create vector of feature means

In [9]:
for c, rows in df.groupby('wine'):
    class_feature_means[c] = (rows.mean())
class_feature_means['class_0'][0]

13.744745762711865

In [13]:
pd.DataFrame(within_class_scatter_matrix.T)

,0,1,2,3,4,5,6,7,8,9,10,11,12
0,45.859182,1.430276,-2.329110,-17.013018,3.138271,4.742176,3.960549,-0.170720,2.934978,43.130146,0.136868,-0.872439,2.141495e+03
1,1.430276,155.320689,3.743391,72.582283,-155.634176,-2.534868,-1.646618,1.780654,3.010536,-45.272514,-7.373182,8.142634,-5.785236e+03
2,-2.329110,3.743391,11.562618,84.728099,120.665125,2.829752,5.244934,1.228605,0.469124,1.713984,0.406392,1.909503,-8.767189e+01
3,-17.013018,72.582283,84.728099,1401.191957,566.621139,19.007669,43.123331,8.210113,16.226898,-17.814143,-1.720402,39.485563,-5.745752e+03
4,3.138271,-155.634176,120.665125,566.621139,31615.110304,100.999304,116.697710,-50.235317,227.569448,310.171993,22.077711,-49.197790,8.334355e+04
5,4.742176,-2.534868,2.829752,19.007669,100.999304,33.472333,28.228731,-1.353757,16.072642,34.821922,-0.256415,10.476063,1.469197e+03
6,3.960549,-1.646618,5.244934,43.123331,116.697710,28.228731,48.073815,-2.671745,22.363830,50.168703,-0.736111,11.875839,5.994972e+02
7,-0.170720,1.780654,1.228605,8.210113,-50.235317,-1.353757,-2.671745,2.084548,-1.512148,-0.264371,0.237752,-1.762567,-8.630433e+01
8,2.934978,3.010536,0.469124,16.226898,227.569448,16.072642,22.363830,-1.512148,43.080265,40.111238,-1.103525,7.427303,2.009406e+03
9,43.130146,-45.272514,1.713984,-17.814143,310.171993,34.821922,50.168703,-0.264371,40.111238,399.861539,-7.174341,-11.603398,1.191639e+04


# STEP 1 - Within class scatter matrix

In [10]:
within_class_scatter_matrix = np.zeros((13,13))

for c, rows in df.groupby('wine'):
    s = np.zeros((13,13))
    rows = rows.drop(columns=['wine'])
    for i, x in rows.iterrows():
        mc = class_feature_means[c]
        r = ((x - mc).values.reshape(13,1))
        res = r.dot(r.T)
        s+=res
    within_class_scatter_matrix+=s

# STEP 1 - Between class scatter matrix

We calculate the between class scatter matrix using the following formula:

$$S_b = \sum_{i=1}^{c}N_i{(m_i-m)(m_i-m)^T}$$

where

$$m_i = \frac{1}{n_i} \sum_{x\in D_i}^{n}{x_k}$$

and

$$m = \frac{1}{n} \sum_{i}^{n}{x_k}$$

In [189]:
# calculate the feature means
feature_means= df.mean()
between_sclass_catter_matrix = np.zeros((13,13))

for c in class_feature_means:
    n = df[df['wine']==c].shape[0]
    
    mc  = class_feature_means[c].values.reshape(13,1)
    m = feature_means.values.reshape(13,1)
    
    between_sclass_catter_matrix += n * (mc-m).dot((mc-m).T)

# Step 2

Compute the eigenvectors and corresponding eigenvalues for the scatter matrices
We solve the generalised eigenvalue problem for

$$S_W^{-1}S_B$$

to obtain the linear discriminants.

In [191]:
eig_val, eig_vec = np.linalg.eig(np.linalg.inv(within_class_scatter_matrix).dot(between_sclass_catter_matrix))

The eigenvectors with the highest eigenvalues carry the most information about the distribution of the data. Thus, we sort the eigenvalues from highest to lowest and select the first $k$ eigenvectors. In order to ensure that the eigenvalue maps to the same eigenvector after sorting, we place them in a temporary array.

In [196]:
pairs = [(np.abs(eig_val[i]), eig_vec[i]) for i in range(len(eig_val))]

In [199]:
sorted(eig_val)

[(-8.881784197001252e-16+0j),
 (-2.6641037342522513e-16-1.1555253947125773e-16j),
 (-2.6641037342522513e-16+1.1555253947125773e-16j),
 (-2.58525572226227e-16+0j),
 (-1.7345934983716223e-16-7.213882930992055e-16j),
 (-1.7345934983716223e-16+7.213882930992055e-16j),
 (-2.814816166461708e-17-5.4411351132571617e-17j),
 (-2.814816166461708e-17+5.4411351132571617e-17j),
 (4.86945776983596e-17+0j),
 (6.521433431033044e-16-8.059240223167847e-17j),
 (6.521433431033044e-16+8.059240223167847e-17j),
 (4.128469045639484+0j),
 (9.081739435042472+0j)]

In [2]:
eig_vec

NameError: name 'eig_vec' is not defined